[slides](https://docs.google.com/presentation/d/19K9nVjuSOCrZGM6lmFeEEarTm2xZwDSiZEIzf-Ywr5o/edit?usp=sharing)

[python-ecdsa docs](https://github.com/warner/python-ecdsa)

# Signing our First Message with ECDSA

In [39]:
from ecdsa import SigningKey, SECP256k1

private_key = SigningKey.generate(curve=SECP256k1)
public_key = private_key.get_verifying_key()

message = b"Execute order 66"
signature = private_key.sign(message)

print(serialize(signature))

b'\x80\x03C@f\xce\x15\xca\xc1i\xb8\x9b\xc17>6N\x1f\xae\xb5c\xca\x99\x05\xebC\xfd\xe8]?\x87\xfda\xb9+Tc\xda!\xe4\xe1\xc4\x94.0\xc15\x1bo2T\xa0L\x030\x14d`\xb7\x13\xf8\xfd\xcd\xcc4\xde5Qq\x00.'


In [40]:
public_key.verify(signature, message)

True

In [11]:
#Counterfiet message produces a BadSignatureError 
message = b"Actually, I want you guys to help the Jedi."
public_key.verify(signature, message)

BadSignatureError: 

# Defining ECDSACoin

* A coin is just a list of transfers, just like with PNGCoin. 
    * Where transfers were photographs of signatures in PNGCoin, they are ECDSA digital signatures in ECDSACoin
* The `public_key` in the last transfer is who owns the coin
* To spend the coin append a new transfer. Use the public key of the person you are sending to, and sign it using your private key.

In [41]:
class Transfer:
    
    def __init__(self, signature, public_key):
        self.signature = signature
        self.public_key = public_key

class ECDSAcoin:

    def __init__(self, transfers):
        self.transfers = transfers
        


In [19]:
# The usual suspects ... 
# SECP256k1 is a detail about the "magical multiplication" used under the covers

bank_private_key = SigningKey.generate(curve=SECP256k1)
bob_private_key = SigningKey.generate(curve=SECP256k1)
alice_private_key = SigningKey.generate(curve=SECP256k1)

bank_public_key = bank_private_key.get_verifying_key()
bob_public_key = bob_private_key.get_verifying_key()
alice_public_key = alice_private_key.get_verifying_key()

In [42]:
from utils import serialize

def issue(public_key):    
    
    message = serialize(public_key)
    signature = bank_private_key.sign(message)
    transfer = Transfer(
        signature=signature,
        public_key=public_key)
    
    coin = ECDSAcoin([transfer])
    return coin

# Validating the First Transfer

In [45]:
def validate(coin):
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    bank_public_key.verify(transfer.signature, message)
    
coin1 = issue(alice_public_key)

#Attempting to validate a legitimate coin
validate(coin1)

In [52]:
#Attempting to validate a forged coin

from ecdsa import BadSignatureError

message = serialize(bob_public_key)
signature = bob_private_key.sign(message)
transfer = Transfer(
    signature=signature,
    public_key=bob_public_key)
    
forged_coin = ECDSAcoin([transfer])

try:
    validate(forged_coin)
except BadSignatureError:
    print('These are not standard Galactic Federation credits. Alert Lord Vader!')

These are not standard Galactic Federation credits. Alert Lord Vader!


# Validating Subsequent Transfers

In [74]:
def transfer_message(signature, public_key):
    return serialize({"previous_signature":signature, "next_owner_public_key": public_key})

def validate(coin):
    
    try:
        #Verifying inital "genesis" transaction
        transfer = coin.transfers[0]
        message = serialize(transfer.public_key)
        bank_public_key.verify(transfer.signature, message)
    
        #Subsequent tx
    
        previous_transfer = coin.transfers[0]
        for next_transfer in coin.transfers[1:]:
            message = transfer_message(previous_transfer.signature, next_transfer.public_key)
            previous_transfer.public_key.verify(next_transfer.signature, message)
    except BadSignatureError:
        print('Fradulant transaction detected!')

In [61]:
def get_owner(coin):
    database = {alice_public_key : 'alice', 
               bob_public_key : 'bob',
               bank_public_key : 'bank'}
    public_key = coin.transfers[-1].public_key
    return database[public_key]

In [80]:
coin1 = issue(alice_public_key)

print("The coin is owned by",get_owner(coin1))

message = transfer_message(coin1.transfers[-1].signature, bob_public_key)
signature = alice_private_key.sign(message)
transfer = Transfer(signature=signature, public_key = bob_public_key)
coin1.transfers.append(transfer)

print("The coin is owned by", get_owner(coin1))
validate(coin1)

#Alice tries to pay her debts off at the bank by signing a transaction from Bob to the bank

message = transfer_message(coin1.transfers[-1].signature, bank_public_key)
signature = alice_private_key.sign(message)
transfer = Transfer(signature=signature, public_key = bob_public_key)
coin1.transfers.append(transfer)

validate(coin1)


The coin is owned by alice
The coin is owned by bob
Fradulant transaction detected!


# Serialization

In [76]:
from utils import from_disk, to_disk

import os

filename = "coin1.ecdsacoin"

print("Does this coin file exist?", os.path.isfile(filename))

Does this coin file exist? False


In [77]:
to_disk(coin1, filename)

In [78]:
print("Does this coin file exist?", os.path.isfile(filename))

Does this coin file exist? True


In [81]:
coin1 = from_disk(filename)
print(coin1)

# The Finished Product

[ecdsacoin.py](ecdsacoin.py)

In [21]:
import ecdsacoin

coin = ecdsacoin.issue(alice_public_key)
coin.validate()

alice_to_bob = Transfer(
    signature=alice_private_key.sign(coin.transfers[0].signature),
    public_key=bob_public_key,
)

coin.transfers.append(alice_to_bob)
coin.validate()